# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn import datasets, tree, ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np

In [3]:
boston = datasets.load_boston()
X, y = boston.data, boston.target

X_train=X[:380]
X_test=X[380:]
y_train=y[:380]
y_test=y[380:]

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [4]:
def L(y, z):
    return z - y

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [5]:
n_trees = range(50)
base_algorithms_list=[tree.DecisionTreeRegressor(max_depth=5, random_state=42) for i in range(50)]
coefficients_list=[0.9 for i in range(50)]

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [6]:
base_algorithms_list  = []
coefficients_list = []
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9

a_0 = tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
a_0.fit(X_train, y_train)
base_algorithms_list.append(a_0)
coefficients_list.append(eta)
s=0

for k in range(49):
    s = L(gbm_predict(X_train), y_train)
    clf = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append(eta)
    m = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5
    mean_sq_err.append(m)
    
    print ('MSE на тестовой выборке:', m)

MSE на тестовой выборке: 5.141717949778777
MSE на тестовой выборке: 5.124049626080035
MSE на тестовой выборке: 5.241709344372247
MSE на тестовой выборке: 5.327156719301596
MSE на тестовой выборке: 5.443683344634833
MSE на тестовой выборке: 5.483452482206295
MSE на тестовой выборке: 5.499420907219651
MSE на тестовой выборке: 5.444470529320642
MSE на тестовой выборке: 5.46055063864994
MSE на тестовой выборке: 5.457445023880429
MSE на тестовой выборке: 5.4605077584473705
MSE на тестовой выборке: 5.4358899991546945
MSE на тестовой выборке: 5.429021196342293
MSE на тестовой выборке: 5.492003306835773
MSE на тестовой выборке: 5.483381945608195
MSE на тестовой выборке: 5.463214301120151
MSE на тестовой выборке: 5.461363250619963
MSE на тестовой выборке: 5.472282089864403
MSE на тестовой выборке: 5.442271584548418
MSE на тестовой выборке: 5.454159056722474
MSE на тестовой выборке: 5.455510181982884
MSE на тестовой выборке: 5.461607177116226
MSE на тестовой выборке: 5.467734887741132
MSE на тес

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [7]:
base_algorithms_list  = []
coefficients_list = []
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9/(1.0 + 0)

a_0 = tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
a_0.fit(X_train, y_train)
base_algorithms_list.append(a_0)
coefficients_list.append(eta)
s=0

for k in range(49):
    s = L(gbm_predict(X_train), y_train)
    clf = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append((eta/(1.0 + (k+1))))
    m2 = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5
    mean_sq_err.append(m2)
    
print ('MSE на тестовой выборке:', m2)

MSE на тестовой выборке: 5.241033584774468


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [8]:
1,3

(1, 3)

In [9]:
base_algorithms_list  = []
coefficients_list = []
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9/(1.0 + 0)

a_0 = tree.DecisionTreeRegressor(max_depth=5 , random_state=42)
a_0.fit(X_train, y_train)
base_algorithms_list.append(a_0)
coefficients_list.append(eta)
s=0

for k in range(70):
    s = L(gbm_predict(X_train), y_train)
    clf = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append((eta/(1.0 + (k+1))))
    m2 = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5
    mean_sq_err.append(m2)
print ('MSE на тестовой выборке:', m2)

MSE на тестовой выборке: 5.260381387672466


In [10]:
base_algorithms_list  = []
coefficients_list = []
mean_sq_err=[]
gbm_predict_lst=[]
eta = 0.9/(1.0 + 0)

a_0 = tree.DecisionTreeRegressor(max_depth=15 , random_state=42)
a_0.fit(X_train, y_train)
base_algorithms_list.append(a_0)
coefficients_list.append(eta)
s=0

for k in range(49):
    s = L(gbm_predict(X_train), y_train)
    clf = tree.DecisionTreeRegressor(max_depth=15, random_state=42)
    clf.fit(X_train, s)
    base_algorithms_list.append(clf)
    coefficients_list.append((eta/(1.0 + (k+1))))
    m2 = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5
    mean_sq_err.append(m2)
print ('MSE на тестовой выборке:', m2)

MSE на тестовой выборке: 7.186720694832733


## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда.

In [11]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
m_linear = (mean_squared_error(y_test, linear_regressor.predict(X_test)))**0.5
print ('RMSE на тестовой выборке с линейной регрессией:', m_linear)

RMSE на тестовой выборке с линейной регрессией: 7.819688142087171
